## 本程序构建feature和label文件链接的csv文件，然后将csv文件保存到“datasets_csv”文件夹内。

## 这里的feature可以是单个，或多个，放置在列表中。名字与对应文件夹相同。

last update: 2022.5.28

contacts：zhaohf@ihep.ac.cn

# 配置环境

## 所需模块

In [1]:
import os
import sys
assert sys.version_info >= (3, 5)
#
import pandas as pd

输出导入模板的版本。

In [2]:
print('python version:',sys.version)
print('panda version:',pd.__version__)

python version: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
panda version: 1.4.2


# 设置参数

## 设置feature和label文件所在的文件夹和相对路径

In [3]:
# the directory name of feature
feature_list = ['chi', 'xmu', 'wt', 'ft', 'bft']
label_list = ['cn1','cr1']
#
#dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_pick'
#dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_treat'
dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab'
path_rela = './../../datasets setup'

## 设置保存csv文件的文件夹路径

In [4]:
dir_csv = './../datasets_csv'
# 保存csv文件中feature、label的文件为绝对路径？True or False
save_csv_file_abspath = False

## 设置csv文件的标记，将出现在csv文件开头

In [5]:
file_index = 'au150_DW'

## 设置统计原始还是已经经过pick的label

In [6]:
# pick_count = False : count all labels
#            = True  : count labels having been picked. (work with check)
pick_count = True
# 如果pick_out = True， 则需要设置如下需要pick out的label情况
dir_check = 'check'
file_checknum = 'check_num.txt'

# 正文

## 确定当前目录和工作目录

In [7]:
path = os.path.abspath(os.curdir)
path

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\neural network module\\jupyternotebook_code'

In [8]:
# 数据集文件夹
path_src = os.path.join(path_rela, dir_src)
if save_csv_file_abspath:
    path_src = os.path.abspath(path_src)
path_src

'./../../datasets setup\\AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab'

In [9]:
# 检查源文件夹是否存在，存在则列出其内部文件夹，不存在则打印错误
exist = os.path.exists(path_src)
if exist:
    list_seed = os.listdir(path_src)
else:
    print()
    print('** Error!! WT tared directory', dir_src, 'is not found!' )
    print()
    sys.exit()
list_seed

['100',
 '1000',
 '1100',
 '1200',
 '1300',
 '1400',
 '1500',
 '1600',
 '1700',
 '1800',
 '1900',
 '200',
 '2000',
 '2100',
 '2200',
 '2300',
 '2400',
 '2500',
 '2600',
 '2700',
 '2800',
 '2900',
 '300',
 '3000',
 '3100',
 '3200',
 '3300',
 '3400',
 '3500',
 '3600',
 '3700',
 '3800',
 '3900',
 '400',
 '4000',
 '500',
 '600',
 '700',
 '800',
 '900',
 'modify.log',
 'modify2.log',
 'pick.log']

In [10]:
# 去除list_seed中不是文件夹的部分
file_del = []
for dir_seed in list_seed:
    path_seed_src = os.path.join(path_src, dir_seed)
    if os.path.isfile(path_seed_src):
        file_del.append(dir_seed)
for file in file_del:
    list_seed.remove(file)
#list_seed

## 将feature和label所在的文件路径制成列表

In [11]:
for feature in feature_list:
    for label in label_list:
        file_csv  = f"{file_index}_{feature}_{label}.csv"
#
        list_path_file_feature = []
        list_path_file_label = []
#
        for dir_seed in list_seed:
#  种子文件夹的路径
            path_dir_seed = os.path.join(path_src,dir_seed)
            path_dir_feature = os.path.join(path_dir_seed, feature)
            path_dir_label = os.path.join(path_dir_seed, label)
#
#  判断“chi_FEFF”等feature文件夹、“coor”等label文件夹在seed文件夹下
            result1 = os.path.isdir(path_dir_feature)
            result2 = os.path.isdir(path_dir_label)
            if result1 and result2:
                list_feature = os.listdir(path_dir_feature)
                list_label = os.listdir(path_dir_label)
#
# 统计是否扣除check文件夹中的label
                result3 = os.path.splitext(list_feature[0])
                result4 = os.path.splitext(list_label[0])
                if pick_count:
                    path_dir_check = os.path.join(path_dir_seed, dir_check)
                    path_file_checknum = os.path.join(path_dir_check, file_checknum)
                    exist_checknum = os.path.exists(path_file_checknum)
                    if exist_checknum:
# csv读取方式
                        with open(path_file_checknum) as f:
                            number_list0 = f.read().splitlines()
                        number_list = []
                        for i, number in enumerate(number_list0):
                            number_list.append(number.strip())
                        for i, number in enumerate(number_list):
                            file_feature_num = feature + '_' + number +result3[1]
                            file_label_num = label + '_' + number + result4[1]
                            list_feature.remove(file_feature_num)
                            list_label.remove(file_label_num)
                    else:
                        print()
                        print(f"** Warning! File '{file_checknum}' in seed {dir_seed} NOT found!")
                        print(f"** All the feature/label will be counted!")
#                   end if exist_checknum
                else:
                    pass
#               end if pick_count
#
                N_feature = len(list_feature)
                N_label = len(list_label)
                if N_feature == N_label :
                    for i in range(N_feature):
                        file_feature = list_feature[i]
                        file_label = list_label[i]
                        path_file_feature = os.path.join(path_dir_feature, file_feature)
                        path_file_label = os.path.join(path_dir_label, file_label)
                        list_path_file_feature.append(path_file_feature)
                        list_path_file_label.append(path_file_label)
                else:
                    print()
                    print('** Error!!', dir_seed, 'has different number of feature/label files !' )
                    sys.exit()
            else:
                print()
                print('** Error!! This seed', dir_seed, 'has no feature/label files !' )
                sys.exit()
#
#将列表保存为csv文件
#
        path_dir_csv = os.path.join(os.curdir, dir_csv)
        os.makedirs(path_dir_csv, exist_ok=True)
#
        path_file_csv = os.path.join(path_dir_csv,file_csv)
        df_samples = pd.DataFrame({'path_feature':list_path_file_feature,'path_label':list_path_file_label})
#
# 将df_samples转化为csv文件
        df_samples.to_csv(path_file_csv)


** Warning! File 'check_num.txt' in seed 1500 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 2200 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 2400 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 300 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 3200 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 3800 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 500 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 600 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 1500 NOT found!
** All the feature/label will be counted!

** Warning! File 'check_num.txt' in seed 2200 NOT found!
** All the feature/label will be counted!

**

In [12]:
df_samples

,path_feature,path_label
0,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
1,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
2,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
3,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
4,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
...,...,...
5941,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
5942,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
5943,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...
5944,./../../datasets setup\AuPd-4000_au150-0_DW_R0...,./../../datasets setup\AuPd-4000_au150-0_DW_R0...


In [13]:
df_samples['path_feature'][0]

'./../../datasets setup\\AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab\\100\\bft\\bft_1.dat'

In [14]:
df_samples['path_label'][0]

'./../../datasets setup\\AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab\\100\\cr1\\cr1_1.dat'